<a href="https://colab.research.google.com/github/ohmreborn/Ai/blob/main/Day%205/%E0%B9%80%E0%B8%99%E0%B8%B7%E0%B9%89%E0%B8%AD%E0%B8%AB%E0%B8%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/cstorm125/choco-raisin

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data.dataloader import default_collate
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import tqdm
import numpy as np

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.TrivialAugmentWide(),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225),)
])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device 

In [ ]:
train_data = datasets.ImageFolder("/content/choco-raisin/images/train", transform=train_transform)
val_data = datasets.ImageFolder("/content/choco-raisin/images/valid", transform=val_transform)

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, collate_fn=lambda x: tuple(x_.to(device) for x_ in default_collate(x)))

In [ ]:
images, labels = next(iter(train_loader))

In [ ]:
with torch.no_grad():
  idx2_class = {v: k for k, v in train_data.class_to_idx.items()}
  fig = plt.figure(figsize=(25, 4))
  for i in range(10):
      image = np.transpose(images.cpu()[i])
      label = idx2_class[labels.cpu().tolist()[i]]
      ax = fig.add_subplot(2, 8, i + 1, xticks=[], yticks=[])
      plt.imshow(image)
      ax.set_title(label)

In [ ]:
model = models.resnet34(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(in_features=512, out_features=len(train_data.classes))
model

In [ ]:
model.to(device)
print(next(model.parameters()).is_cuda)

In [ ]:
cross_entropy = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=2e-3)

In [ ]:
epoch = 1
train_length = len(train_loader)
val_length = len(val_loader)
for i in range(epoch):
  loss_train = []
  loss_eval = []
  model.train()
  with tqdm.auto.tqdm(total=train_length, desc="train: ") as bar:
    for images,label in train_loader:
      optimizer.zero_grad()
      output = model(images)
      loss = cross_entropy(output,label)
      loss.backward() # คำนวณ gradient จาก loss ที่ได้
      optimizer.step()
      with torch.no_grad():
        loss_train.append(loss.item())
        bar.update(1)
        bar.set_description(f"loss train={sum(loss_train)/len(loss_train):.2f}")
      
  model.eval()
  with tqdm.auto.tqdm(total=val_length, desc="validation: ") as bar:
    for images,label in val_loader:
      output = model(images)
      with torch.no_grad():
        loss = cross_entropy(output,label)
        loss_eval.append(loss.item())
        bar.update(1)  #
        bar.set_description(f"loss val={sum(loss_eval)/len(loss_eval):.2f}")

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support# คำนวณหา classification report สำหรับ validation set
y_pred, y_true = [], []
model.eval()
for images, labels in val_loader:
    pred = model(images)
    yp = pred.argmax(dim=1).tolist()
    yt = labels.tolist()
    y_pred.extend(yp)
    y_true.extend(yt)
print(classification_report(y_true, y_pred))

In [ ]:
print("Accuracy on validation set = {}".format(
    accuracy_score(y_true, y_pred))
)